In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip install tensorflow-hub
!pip install numpy==1.16.1
#!pip install keras==2.1.2
!pip install git+https://github.com/LanceNorskog/deep_meter_2#egg=deepmeter
#!pip install sortedcontainer
!wget  https://raw.githubusercontent.com/LanceNorskog/deep_meter_2/master/haiku_5.txt
!cut -f2 < haiku_5.txt | sort | uniq > haiku_5_short.txt
!wc -l haiku_5*.txt

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
--2019-07-04 02:10:09--  https://raw.githubusercontent.com/LanceNorskog/deep_meter_2/master/haiku_5.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29054274 (28M) [text/plain]
Saving to: ‘haiku_5.txt.7’

haiku_5.txt.7       100%[===================>]  27.71M   166MB/s    in 0.2s    

2019-07-04 02:10:11 (166 MB/s) - ‘haiku_5.txt.7’ saved [29054274/29054274]

   95631 haiku_5_short.txt
  673680 haiku_5.txt
  769311 total


In [2]:

from __future__ import print_function
import math
import pickle
import json
import os
import glob
import time

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import keras.backend as K
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, Sequential, load_model
from keras import layers 
from keras import metrics
from keras.datasets import reuters
from keras.preprocessing import text

from sklearn.model_selection import train_test_split

from cmu.syllables_cmu import syllables as word2sylls
print(word2sylls['therefore'])

# number of total samples to use
max_data = 100000
# cut texts after this number of words
# number of output syllables in short haiku
max_features = 16000
# longest output sentence
maxlen = 5
batch_size = 32
deduplicate_haiku=True
model_base="/content/gdrive/My Drive/Colab Notebooks/haiku_5_"
model_file=model_base + "{}.h5".format(int(time.time()))
for file in glob.glob(model_base + '*'):
    print('found: ', file)
    model_file=file

print(model_file)


Using TensorFlow backend.


['DH EH R', 'F AO R']
found:  /content/gdrive/My Drive/Colab Notebooks/haiku_5_1562204555.h5
/content/gdrive/My Drive/Colab Notebooks/haiku_5_1562204555.h5


In [3]:
!date
big_sylls = set()
for index, word in enumerate(word2sylls):
    sylls = word2sylls[word]
    for syll in sylls:
        syll = syll.replace(' ', '').lower()
        big_sylls.add(syll)

syll2idx = {}
for index, syll in enumerate(big_sylls):
    syll2idx[syll] = index
idx2syll = [0] * len(syll2idx)
for index, syll in enumerate(syll2idx):
    idx2syll[index] = syll

print(syll2idx['dhehr'], idx2syll[1])
print('# features: ', len(idx2syll))


Thu Jul  4 02:10:18 UTC 2019
4718 waabst
# features:  15086


In [4]:
big_text = []
big_haiku = []
big_data = []
big_data_file = "haiku_5.txt"
with open(big_data_file) as f:
    last_haiku = ''
    for line in f.readlines():
        _parts = line.strip().split('\t')
        _text = _parts[0]
        _haiku = _parts[1]
        _sylls = []
        if deduplicate_haiku and _haiku == last_haiku:
            continue
        for word in text.text_to_word_sequence(_haiku):
            if word in word2sylls:
                for syll in word2sylls[word]:
                    _sylls.append(syll.replace(' ', '').lower())
        if len(_sylls) != 5:
            continue
        _data = np.zeros((5), dtype='int32')
        for j in range(5):
             _data[j] = syll2idx[_sylls[j]]
        big_text.append(_text)
        big_haiku.append(_haiku)
        big_data.append(_data)
        #if len(big_text) == max_data * 2:
        #    break

big_text = np.array(big_text)
big_haiku = np.array(big_haiku)
big_data = np.array(big_data)
print('{} -> {} : {}'.format(big_text[0], big_haiku[0], big_data[0]))

shuffle = np.arange(len(big_text))
print(shuffle)
np.random.shuffle(shuffle)
shuffle = shuffle[0:max_data]
big_text = big_text[shuffle]
big_haiku = big_haiku[shuffle]
big_data = big_data[shuffle]
print('{} -> {} : {}'.format(big_text[0], big_haiku[0], big_data[0]))

print('Full length clauses: ', len(big_text))

a white sink and door -> a white sink and door : [10891 12555 10804  9657  2156]
[     0      1      2 ... 668545 668546 668547]
on the blue water -> on the blue water : [ 3338  6429 12032  8309  8805]
Full length clauses:  100000


In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [6]:
big_data = np.expand_dims(big_data, -1)
(x_train, x_test, y_train, y_test) = train_test_split(big_text, big_data)
x_train = x_train[0:(len(x_train) // batch_size) * batch_size]
y_train = y_train[0:(len(y_train) // batch_size) * batch_size]
x_test = x_test[0:(len(x_test) // batch_size) * batch_size]
y_test = y_test[0:(len(y_test) // batch_size) * batch_size]

print(x_train[0], y_train[0])
print(x_test[0], y_test[0])
print(x_train[-1], y_train[-1])
print(x_test[-1], y_test[-1])

def get_lstm(size, return_sequences=True):
    return layers.CuDNNLSTM(size, return_sequences=return_sequences)

#x_train = np.array(x_train)
#x_test = np.array(x_test)
#y_train = np.expand_dims(y_train, -1)
#y_test = np.expand_dims(y_test, -1)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print(y_test[0][0])



of a computer [[10349]
 [10891]
 [ 6980]
 [ 3855]
 [ 8805]]
yellow swirl cookies [[ 8964]
 [11411]
 [14000]
 [  111]
 [14955]]
next to a painting [[10528]
 [14892]
 [10891]
 [ 6027]
 [  189]]
flying through the air [[13963]
 [ 4836]
 [ 9816]
 [ 6429]
 [ 3703]]
x_train shape: (74976,)
x_test shape: (24992,)
y_train shape: (74976, 5, 1)
y_test shape: (24992, 5, 1)
[8964]


In [0]:
# https://github.com/philipperemy/keras-snail-attention/blob/master/attention.py
# Do these Dense layers need activation tanh?
# https://www.d2l.ai/chapter_attention-mechanism/attention.html
# k, q have attention, v does not?
class AttentionBlock(layers.Layer):

    def __init__(self, dims, k_size, v_size, seq_len=None, **kwargs):
        self.k_size = k_size
        self.seq_len = seq_len
        self.v_size = v_size
        self.dims = dims
        self.sqrt_k = math.sqrt(k_size)
        self.keys_fc = None
        self.queries_fc = None
        self.values_fc = None
        super(AttentionBlock, self).__init__(**kwargs)

    def build(self, input_shape):
        # https://stackoverflow.com/questions/54194724/how-to-use-keras-layers-in-custom-keras-layer
        self.keys_fc = layers.Dense(self.k_size)
        self.keys_fc.build((None, self.dims))
        self._trainable_weights.extend(self.keys_fc.trainable_weights)

        self.queries_fc = layers.Dense(self.k_size)
        self.queries_fc.build((None, self.dims))
        self._trainable_weights.extend(self.queries_fc.trainable_weights)

        self.values_fc = layers.Dense(self.v_size)
        self.values_fc.build((None, self.dims))
        self._trainable_weights.extend(self.values_fc.trainable_weights)

    def call(self, inputs, **kwargs):
        # check that the implementation matches exactly py torch.
        keys = self.keys_fc(inputs)
        queries = self.queries_fc(inputs)
        values = self.values_fc(inputs)
        logits = K.batch_dot(queries, K.permute_dimensions(keys, (0, 2, 1)))
        mask = K.ones_like(logits) * np.triu((-np.inf) * np.ones(logits.shape.as_list()[1:]), k=1)
        logits = mask + logits
        probs = layers.Softmax(axis=-1)(logits / self.sqrt_k)
        read = K.batch_dot(probs, values)
        output = K.concatenate([inputs, read], axis=-1)
        return output

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] += self.v_size
        return tuple(output_shape)



In [0]:
def sparse_categorical_accuracy(y_true, y_pred):
    # reshape in case it's in shape (num_samples, 1) instead of (num_samples,)
    if K.ndim(y_true) == K.ndim(y_pred):
        y_true = K.squeeze(y_true, -1)
    # convert dense predictions to labels
    y_pred_labels = K.argmax(y_pred, axis=-1)
    y_pred_labels = K.cast(y_pred_labels, K.floatx())
    return K.cast(K.equal(y_true, y_pred_labels), K.floatx())

def sparse_categorical_accuracy_per_sequence(y_true, y_pred):
    # reshape in case it's in shape (num_samples, 1) instead of (num_samples,)
    if K.ndim(y_true) == K.ndim(y_pred):
        y_true = K.squeeze(y_true, -1)
    # convert dense predictions to labels
    y_pred_labels = K.argmax(y_pred, axis=-1)
    y_pred_labels = K.cast(y_pred_labels, K.floatx())
    return K.min(K.cast(K.equal(y_true, y_pred_labels), K.floatx()), axis=-1)

def sparse_temporal_top_k_categorical_accuracy(y_true, y_pred, k=5):
    original_shape = K.shape(y_true)
    y_true = K.reshape(y_true, (-1, K.shape(y_true)[-1]))
    y_pred = K.reshape(y_pred, (-1, K.shape(y_pred)[-1]))
    top_k = K.in_top_k(y_pred, K.cast(K.max(y_true, axis=-1), 'int32'), k)
    return K.reshape(top_k, original_shape[:-1])

def sparse_temporal_top_k_categorical_accuracy_per_sequence(y_true, y_pred, k=5):
    original_shape = K.shape(y_true)
    y_true = K.reshape(y_true, (-1, K.shape(y_true)[-1]))
    y_pred = K.reshape(y_pred, (-1, K.shape(y_pred)[-1]))
    top_k = K.in_top_k(y_pred, K.cast(K.max(y_true, axis=-1), 'int32'), k)
    perfect = K.min(K.cast(top_k, 'int32'), axis=-1)
    return perfect #K.expand_dims(perfect, axis=-1)

def sparse(y_true, y_pred):
    return sparse_categorical_accuracy(y_true, y_pred)
def sparse1(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy(y_true, y_pred, k=1)
def perfect(y_true, y_pred):
    return sparse_categorical_accuracy_per_sequence(y_true, y_pred)
def perfect1(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy_per_sequence(y_true, y_pred, k=1)
def sparse5(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy(y_true, y_pred, k=5)
def perfect5(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy_per_sequence(y_true, y_pred, k=5)
def fscore(y_true, y_pred):
    recall = K.mean(sparse_categorical_accuracy(y_true, y_pred))
    precision = K.mean(sparse_categorical_accuracy_per_sequence(y_true, y_pred))
    return 2 * ((recall * precision)/(recall + precision))


units_k=embed_size
units_v=embed_size//3
units=512

metric_list = [sparse, perfect]
metric_names = ['sparse', 'perfect']

input_text = layers.Input(shape=(1,), dtype=tf.string)
x = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,), name='TF-Hub')(input_text)
x = layers.RepeatVector(maxlen)(x)
x = layers.Dropout(0.5)(x)
if False:
    x = AttentionBlock(embed_size, k_size=units_k, v_size=units_v)(x)
x = get_lstm(units, return_sequences=True)(x)
#x = layers.Dropout(0.5)(x)
#x = layers.Dense(units*2, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(max_features, activation='softmax')(x)


model = Model(inputs=[input_text], outputs=x)
model.compile('adam', 'sparse_categorical_crossentropy', metrics=metric_list)
model.summary()

# try using different optimizers and different optimizer configs

print('Train...')
history = None
use_saved_model=False
if not use_saved_model or not os.path.exists(model_file):
  with tf.Session() as session:
    model_file=model_base + "{}.h5".format(int(time.time()))
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())

    history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          callbacks=[EarlyStopping(monitor='val_perfect', mode='max', verbose=1, patience=2)],
          verbose=2,
          validation_data=[x_test, y_test])
    #model.save_weights(model_file)
#              ModelCheckpoint(model_file, monitor='val_perfect', save_best_only=True, save_weights_only=True, mode='max', verbose=1)],


W0704 02:10:34.683193 140670892853120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0704 02:10:34.684583 140670892853120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0704 02:10:35.577178 140670892853120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0704 02:10:35.587827 140670892853120 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a futur

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
TF-Hub (Lambda)              (None, 512)               0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 512)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 512)            0         
_________________________________________________________________
attention_block_1 (Attention (None, 5, 682)            612522    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 5, 512)            2449408   
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 512)            0         
__________

W0704 02:10:39.580710 140670892853120 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 74976 samples, validate on 24992 samples
Epoch 1/10


In [0]:

if history != None:
  names = metric_names + ['loss']
  # summarize history for accuracy
  for m in names:
      #plt.plot(history.history[m])
      plt.plot(history.history['val_' + m])
  plt.title('model accuracy')
  plt.xlabel('epoch')
  plt.legend(names, loc='upper right')
  plt.show()

In [0]:
x_short = x_test[0:1000]
y_short = y_test[0:1000]
predicts = None

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  #model = load_model(model_file)  
  model.load_weights(model_file)  
  #eval = model.evaluate(x_test, y_test)
  #print('model.evaluate on val holdout: ' ,model.metrics_names, eval)
  print('history: ', history)
  predicts = model.predict(x_short, batch_size=32)
  print('shape: {}'.format(predicts.shape))

print(len(predicts[0]))
print(len(predicts[0][0]))
print(predicts[0])

In [0]:
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  #model = load_model(model_file)  
  model.load_weights(model_file)  
  eval_small = model.evaluate(big_haiku, big_data)
  print('model.evaluate on haiku clauses: ' ,model.metrics_names, eval_small)
  print('history: ', history)
  eval_big = model.evaluate(big_text, big_data)
  print('model.evaluate on long clauses: ' ,model.metrics_names, eval_big)
  print('history: ', history)
  biglen = len(big_text)
  #for i in range(0, len(big_text), batch_size):
  #  predicts = model.predict(big_text[i:i + batch_size], batch_size=batch_size)
  #  print('shape: {}'.format(predicts.shape))


In [0]:
def printhistory(history):
    print(','.join(model.metrics_names))
    out = ''
    for i in range(len(model.metrics_names)):
        out += str(history[i]) + ','
    print(out[0:-1])
    
printhistory(eval_small)
printhistory(eval_big)

In [0]:
def report(data, prediction):
    def match(data, prediction):
        good = 0
        top5 = 0
        count = 0
        for i in range(len(data)):
            topind = np.argsort(prediction[i])
            if data[i][0] == topind[-1]:
                good += 1
            topind = topind[-5:len(topind)]
            for j in range(5):
                if data[i][0] == topind[j]:
                    top5 += 1
                    break
            count += 1
        return (good, top5, count)

    _sparse = 0.0
    _perfect = 0.0
    _sparse5 = 0.0
    _perfect5 = 0.0
    _total = 0
    for n in range(len(data)):
        #print(len(short[n]))
        (good, top5, count) = match(data[n], predicts[n])
        if count == 0:
            continue
        _sparse += good/count
        _sparse5 += top5/count
        if good == count:
            _perfect += 1  
        if top5 == count:
            _perfect5 += 1
        _total += 1
    return {'sparse':_sparse/_total, 'perfect': _perfect/_total, 'sparse5': _sparse5/_total, 'perfect5': _perfect5/_total}


In [0]:
bigbatch = batch_size * 32
big_text = np.array(big_text)
big_haiku = np.array(big_haiku)
text5arr = []
haiku5mean = None
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  #model = load_model(model_file)  
  model.load_weights(model_file)  
  predicts = model.predict(big_haiku[0: bigbatch], batch_size=bigbatch)
  rep = report(big_data[0: bigbatch], predicts)
  print("short {}".format(rep))
  haiku5mean = rep['perfect5']
  biglen = len(big_text)
  for i in range(0, biglen, bigbatch):
    predicts = model.predict(big_text[i:i + bigbatch], batch_size=bigbatch)
    rep = report(big_data[i:i + bigbatch], predicts)
    print("[{}] {}".format(i, rep))
    text5arr.append(rep['perfect5'])

text5mean = np.mean(np.array(text5arr))

In [0]:
val5arr = []
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  #model = load_model(model_file)  
  model.load_weights(model_file)  
  biglen = len(x_test)
  for i in range(0, biglen, bigbatch):
    predicts = model.predict(x_test[i:i + bigbatch], batch_size=bigbatch)
    rep = report(y_test[i:i + bigbatch], predicts)
    print("[{}] {}".format(i, rep))
    val5arr.append(rep['perfect5'])

val5mean = np.mean(np.array(val5arr))

In [0]:
print('Perfect5 for all haiku lines: {}, all mscoco lines: {}, validation mscoco: {}'.format(haiku5mean, text5mean, val5mean))